Application of the abstract theory
===
$\DeclareMathOperator{\opdiv}{div}$


We consider now second order equations with mixed boundary conditions, and variable coefficients:

Find $u$ such that
\begin{eqnarray*}
-\opdiv \lambda(x) \nabla u & = & f \quad \; \; \; \text{ in } \Omega \\
u & = & u_D \quad \text{ on } \Gamma_D \\
\lambda \frac{\partial u}{\partial n} & = & g \quad \; \; \; \text{ on } \Gamma_N
\end{eqnarray*}

Introduce $\sigma = \lambda \nabla u$ to obtain the system

$$
\lambda^{-1} \sigma - \nabla u = 0 \quad \quad \opdiv \sigma = -f
$$

and boundary conditions

$$
u = u_D \; \; \text{ on } \Gamma_D \qquad \sigma \cdot n = g \; \; \text{ on } \Gamma_N
$$

We test the first equation by $\tau$, the second by $v$ and get

$$
\begin{array}{ccccll}
\int \lambda^{-1} \sigma \tau & - & \int \nabla u \tau & = & 0 & \forall \, \tau \\
\int \opdiv \sigma \, v & & & = & - \int f v & \forall \, v
\end{array}
$$

To get a mixed system with the same bilinear-form at the lower left and upper right blocks, we have to integrate by parts either one of the terms. 

Dual mixed formulation
----

Integration by parts of the upper right term we get

$$
- \int_\Omega \nabla u \tau = 
\int_\Omega u \opdiv \tau - \int_{\Gamma_D} u \, \tau_n - \int_{\Gamma_N} u \, \tau_n
$$

Now, we can plug in the boundary condition $u = u_D$ for the Dirichlet boundary, and move the term to the right hand side. On the Neumann boundary, we have to restrict the test-function to $\tau_n = 0$. Neumann boundary conditions become now essential ones.

The problem is now formulated as:

Find $\sigma \in H(\opdiv)$ and $u \in L_2$ such that
$\sigma_n = g$ on $\Gamma_N$ and 

$$
\begin{array}{ccccll}
\int \lambda^{-1} \sigma \tau & + & \int \opdiv \tau \, u  & = & \int_{\Gamma_D} u_D \tau_n & \forall \, \tau, \; \tau_n = 0 \text{ on } \Gamma_N \\
\int \opdiv \sigma \, v & & & = & - \int f v & \forall \, v
\end{array}
$$

Note that the role of essential and natural boundary conditions is now swapped.


There are some issues left to verify:
* Is it allowed to take boundary values of $H(\opdiv)$ functions ? 
* What kind of essential Neumann boundary data can we prescribe ? 
* Is $\int u_D \tau_n$ a continuous linear form ?


In [1]:
from ngsolve import *
import netgen.gui
%gui tk

from netgen.geom2d import *
geo = SplineGeometry()
geo.AddRectangle( (0,0), (1,1), leftdomain=1, rightdomain=0, 
                 bcs=['b','r','t','l'])
geo.AddCircle( (0.3,0.7), 0.1, leftdomain=2, rightdomain=1)
geo.AddRectangle ( (0.2,0.2), (0.9,0.3), leftdomain=3, rightdomain=1)
geo.SetMaterial(1, "air")
geo.SetMaterial(2, "source")
geo.SetMaterial(3, "bar")

mesh = Mesh(geo.GenerateMesh(maxh=0.03))
mesh.Curve(3)
lamvalues = { "air" : 1, "bar" : 100, "source" : 1 }

In [2]:
fes = H1(mesh, order=3, dirichlet="b|r")
u = fes.TrialFunction()
v = fes.TestFunction()

lam = CoefficientFunction( 
    [lamvalues[mat] for mat in mesh.GetMaterials()])

a = BilinearForm(fes)
a += SymbolicBFI (lam * grad(u) * grad(v))
f = LinearForm(fes)
f += SymbolicLFI (1 * v, definedon=mesh.Materials("source"))
a.Assemble()
f.Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu, mesh, "stdtemp")
Draw (-lam*grad(gfu), mesh, "stdflux")

In [3]:
Sigma = HDiv(mesh, order=2, dirichlet="t|l")
V = L2(mesh, order=1)
X = FESpace([Sigma,V])

sigma, u = X.TrialFunction()
tau, v = X.TestFunction()

invlam = CoefficientFunction( 
    [1/lamvalues[mat] for mat in mesh.GetMaterials()])

a = BilinearForm(X)
a += SymbolicBFI(invlam*sigma*tau + div(sigma)*v + div(tau)*u)
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI (1 * v, definedon=mesh.Materials("source"))
f.Assemble()

gfu = GridFunction(X)
gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec
Draw (gfu.components[0], mesh, "mixedflux")
Draw (-gfu.components[1], mesh, "mixedtemp")

We observe that the normal component of the flux is continuous arcross discontinuous coefficients. As we will see, this is a property of the function space $H(\opdiv)$.

Primal mixed formulation
---
We can also integrate the lower left term by part. This leads to the formulation

Find $u \in H^1$ and $\sigma \in [L_2]^d$ such that $u = u_D$ on $\Gamma_D$ and

$$
\begin{array}{ccccll}
\int \lambda^{-1} \sigma \tau & - & \int \tau \nabla u   & = & 0 & \forall \, \tau \\
-\int \sigma \nabla v & & & = & - \int f v - \int_{\Gamma_N} g v & \forall \, v, \; \; v = 0 \text{ on } \Gamma_D
\end{array}
$$

This formulation is very close to the primal method. 